In [1]:
from IPython.display import Markdown
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd


# Define PolicyEngine's color palette
BLACK = "#000000"
BLUE_LIGHT = "#D8E6F3"
BLUE_PRIMARY = "#2C6496"
DARK_BLUE_HOVER = "#1d3e5e"
DARK_GRAY = "#616161"
DARKEST_BLUE = "#0C1A27"
GRAY = "#808080"
LIGHT_GRAY = "#F2F2F2"
MEDIUM_DARK_GRAY = "#D2D2D2"
MEDIUM_LIGHT_GRAY = "#BDBDBD"
WHITE = "#FFFFFF"

In [2]:
from policyengine_us import Simulation
from policyengine_core.reforms import Reform

reform = Reform.from_dict({
  "gov.irs.social_security.taxability.rate.base": {
    "2024-01-01.2100-12-31": 0.85
  },
  "gov.irs.social_security.taxability.threshold.base.main.JOINT": {
    "2024-01-01.2100-12-31": 0
  },
  "gov.irs.social_security.taxability.threshold.base.main.SINGLE": {
    "2024-01-01.2100-12-31": 0
  },
  "gov.irs.social_security.taxability.threshold.base.main.SEPARATE": {
    "2024-01-01.2100-12-31": 0
  },
  "gov.irs.social_security.taxability.threshold.base.main.SURVIVING_SPOUSE": {
    "2024-01-01.2100-12-31": 0
  },
  "gov.irs.social_security.taxability.threshold.base.main.HEAD_OF_HOUSEHOLD": {
    "2024-01-01.2100-12-31": 0
  }
}, country_id="us")


situation = {
  "people": {
    "you": {
      "age": {
        "2026": 70
      },
      "social_security_retirement": {
        "2026": 30000
      }
    },
    "your partner": {
      "age": {
        "2026": 70
      }
    }
  },
  "families": {
    "your family": {
      "members": [
        "you",
        "your partner"
      ]
    }
  },
  "marital_units": {
    "your marital unit": {
      "members": [
        "you",
        "your partner"
      ]
    }
  },
  "tax_units": {
    "your tax unit": {
      "members": [
        "you",
        "your partner"
      ]
    }
  },
  "spm_units": {
    "your household": {
      "members": [
        "you",
        "your partner"
      ]
    }
  },
  "households": {
    "your household": {
      "members": [
        "you",
        "your partner"
      ],
      "state_name": {
        "2026": "FL"
      }
    }
  },
  "axes": [
    [
      {
        "name": "employment_income",
        "count": 401,
        "min": 0,
        "max": 200000
      }
    ]
  ]
}

# Calculate reform net income
simulation_reform = Simulation(
    reform=reform,
    situation=situation,
)
reform_net_income = simulation_reform.calculate("household_net_income", 2026)

# Calculate baseline net income
simulation_baseline = Simulation(
    situation=situation,
)
baseline_net_income = simulation_baseline.calculate("household_net_income", 2026)

# Calculate change in net income
change_in_net_income = reform_net_income - baseline_net_income

print(f"Employment income range: 401 points from $0 to $200,000 in $500 increments")
print(f"Change in net income (first 10 values): {change_in_net_income[:10]}")
print(f"Change in net income (last 10 values): {change_in_net_income[-10:]}")

c:\Users\dtsax\envs\pe\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Employment income range: 401 points from $0 to $200,000 in $500 increments
Change in net income (first 10 values): [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Change in net income (last 10 values): [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [6]:
# Create employment income array with $500 increments (401 points from $0 to $200,000)
employment_income = [i * 500 for i in range(401)]

# Create DataFrame with the actual reform output
df = pd.DataFrame(
    {
        "Employment Income": employment_income,
        "Change in Net Income": change_in_net_income,
    }
)

# Create the line graph
fig = px.line(
    df,
    x="Employment Income",
    y="Change in Net Income",
    color_discrete_sequence=[BLUE_PRIMARY],
    title="Change in Net Income for an Elderly Married Couple with <br>$30,000 in Social Security Benefits (Reform Impact)",
).update_layout(
    font=dict(family="Roboto Serif"),
    xaxis_title="Employment Income ($)",
    yaxis_title="Change in Net Income ($)",
    xaxis_tickformat=",",
    yaxis_tickformat=",",
    font_color=BLACK,
    margin={"l": 50, "r": 50, "b": 100, "t": 100, "pad": 4},
    annotations=[
        {
            "x": 1,
            "y": -0.25,
            "xref": "paper",
            "yref": "paper",
            "text": "Source: PolicyEngine US",
            "showarrow": False,
            "font": {"family": "Roboto Serif", "size": 10, "color": DARK_GRAY},
        }
    ],
)

# Try multiple display methods
try:
    fig.show()
except:
    try:
        import plotly.io as pio
        pio.show(fig, renderer="browser")
    except:
        # Fallback to basic display
        display(fig)

In [7]:
# Create separate graph showing both reform and baseline net income
employment_income = [i * 500 for i in range(401)]

# Create DataFrame with both reform and baseline data
df_comparison = pd.DataFrame({
    "Employment Income": employment_income,
    "Reform Net Income": reform_net_income,
    "Baseline Net Income": baseline_net_income,
})

# Melt the dataframe for plotting multiple lines
df_melted = df_comparison.melt(
    id_vars=["Employment Income"],
    value_vars=["Reform Net Income", "Baseline Net Income"],
    var_name="Scenario",
    value_name="Net Income"
)

# Create the line graph with both scenarios
fig_comparison = px.line(
    df_melted,
    x="Employment Income",
    y="Net Income",
    color="Scenario",
    color_discrete_map={
        "Reform Net Income": BLUE_PRIMARY,
        "Baseline Net Income": DARK_GRAY
    },
    title="Net Income Comparison: Reform vs Baseline for Elderly Married Couple <br>with $30,000 in Social Security Benefits",
).update_layout(
    font=dict(family="Roboto Serif"),
    xaxis_title="Employment Income ($)",
    yaxis_title="Net Income ($)",
    xaxis_tickformat=",",
    yaxis_tickformat=",",
    font_color=BLACK,
    margin={"l": 50, "r": 50, "b": 100, "t": 100, "pad": 4},
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    ),
    annotations=[
        {
            "x": 1,
            "y": -0.25,
            "xref": "paper",
            "yref": "paper",
            "text": "Source: PolicyEngine US",
            "showarrow": False,
            "font": {"family": "Roboto Serif", "size": 10, "color": DARK_GRAY},
        }
    ],
)

# Try multiple display methods
try:
    fig_comparison.show()
except:
    try:
        import plotly.io as pio
        pio.show(fig_comparison, renderer="browser")
    except:
        # Fallback to basic display
        display(fig_comparison)